In [1]:
import pandas as pd
import numpy as np
import autosklearn.classification
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from ml_metrics import quadratic_weighted_kappa

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
all_data = pd.DataFrame()
all_data = train.append(test)

In [4]:
features = all_data.columns
features = features.drop('Id')
features = features.drop('Response')

all_data.fillna(-1, inplace=True)
all_data['Product_Info_2'] = pd.factorize(all_data['Product_Info_2'])[0]

In [5]:
train = all_data[all_data['Response']>0]
test = all_data[all_data['Response']<1]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train[features], train['Response'], test_size=0.1, random_state=42)

X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
y_train = y_train.values-1
y_test = y_test.values-1

#X_train = train[features].as_matrix()
#y_train = train['Response'].values-1
#X_test = test[features].as_matrix()


#print test[features].columns



#print train[features].columns

#the trouble is that it always predicts 0... why is this?

## key params:
###### time_left_for_this_task=seconds (default=3600)
###### per_run_time_limit=seconds (default=360)
###### initial_configurations_via_metalearning : int, optional (default=25)
###### ensemble_size : int, optional (default=50)
###### ensemble_nbest : int, optional (default=50)
###### seed : int, optional (default=1)
###### ml_memory_limit : int, optional (3000)
Memory limit for the machine learning algorithm. If the machine learning algorithm allocates tries to allocate more memory, its evaluation will be stopped.
###### include_estimators : dict, optional (None)
If None all possible estimators are used. Otherwise specifies set of estimators to use
###### include_preprocessors : dict, optional (None)
If None all possible preprocessors are used. Otherwise specifies set of preprocessors to use
###### resampling_strategy : string, optional (‘holdout’)
how to to handle overfitting, might need ‘resampling_strategy_arguments’
‘holdout’: 66:33 (train:test) split
‘holdout-iterative-fit’: 66:33 (train:test) split, calls iterative fit where possible
‘cv’: crossvalidation, requires ‘folds’
‘nested-cv’: crossvalidation, requires ‘outer-folds, ‘inner-folds’
‘partial-cv’: crossvalidation, requires ‘folds’ , calls iterative fit where possible

In [8]:
%%time
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60, resampling_strategy='holdout')

#automl_cv = cross_val_score(automl, X_train, y_train, cv=10).mean()
#print automl_cv

automl.fit(X_train, y_train)



#print(automl.score(X_test,y_test))

[INFO] [2016-02-05 15:32:22,229:AutoML(1):5beccaef9a1a76d29b011fa79fd094b6] Start calculating metafeatures for 5beccaef9a1a76d29b011fa79fd094b6
[INFO] [2016-02-05 15:32:22,314:AutoML(1):5beccaef9a1a76d29b011fa79fd094b6] Calculating Metafeatures (categorical attributes) took  0.09
[INFO] [2016-02-05 15:32:24,001:AutoML(1):5beccaef9a1a76d29b011fa79fd094b6] Calculating Metafeatures (encoded attributes) took  1.69sec
[INFO] [2016-02-05 15:32:24,575:AutoML(1):5beccaef9a1a76d29b011fa79fd094b6] Time left for 5beccaef9a1a76d29b011fa79fd094b6 after finding initial configurations: 54.35sec
[INFO] [2016-02-05 15:32:24,583:autosklearn.util.smac] Start SMAC with 54.35sec time left
[INFO] [2016-02-05 15:32:24,584:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_2556_7962/5beccaef9a1a76d29b011fa79fd094b6.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0 --intensification-percentage 0.5 --validation false --i

/home/patrick/anaconda2/envs/autolearn/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1136: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)
/home/patrick/anaconda2/envs/autolearn/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [19]:
#print automl.score(X_test, y_test)
preds = automl.predict(X_test)

#submission = pd.DataFrame({
#        "Id": df_test["Id"],
#        "Response": y_pred2
#    })
#submission.to_csv('prudential.csv', index=False)

In [20]:
#what i need to do is assume this is a probability distribution and then take the column position of the max value... maybe
#for i in range(preds.shape[0]):
#    print preds[i][0]

index_max=[]
for i in range(preds.shape[0]):
    index_max.append(preds[i].argmax())
    
#print index_max, len(index_max)



#after this stage remember how to output into a csv file with Id and Response
#print zip(index_max, y_test)

#index_max = np.asarray(index_max)
#index_max = index_max+1
#y_test = y_test+1

In [30]:
print quadratic_weighted_kappa(index_max, y_test)

from scipy.optimize import fmin_powell

x0 = [-0.5]
fmin_powell(-quadratic_weighted_kappa(index_max, y_test), x0)

#default settings
#score for 60 second time window (which is really more like 2m 50s) [QWK]: 0.5213
#score for 600 second time window .........................14m 22s) [QWK]: 0.5580

#ensemble_size=100
#score for 60 second time window ...........................2m 27s) [QWK]: 0.1180

#resampling_strategy='holdout'
#score for 60 second time window ...........................2m 37s) [QWK]: 0.5288
#score for 60 second time window............................2m 56s) [QWK]: 0.5288 - replicate above

#resampling_strategy='holdout' + offset function -- *won't work* -- offset assumes 


#resampling_strategy='holdout', per_run=720
#score for 28800 (8hr) time window.......................8h 8m 23s) [QWK]: 0.5638





#so reinstall autosklearn and then pass the results to another file so
#that we can run NN and XGBoost



0.521307712719


TypeError: 'float' object is not callable

In [93]:
y_test_vals = [int(i) for i in y_test]
zip(index_max, y_test_vals)

[(0, 6),
 (7, 7),
 (4, 1),
 (7, 3),
 (4, 5),
 (6, 1),
 (7, 7),
 (7, 6),
 (7, 7),
 (4, 0),
 (7, 7),
 (7, 7),
 (0, 0),
 (5, 2),
 (5, 4),
 (4, 4),
 (7, 7),
 (5, 4),
 (5, 6),
 (5, 1),
 (4, 4),
 (7, 7),
 (1, 1),
 (7, 7),
 (5, 5),
 (5, 5),
 (7, 4),
 (5, 3),
 (6, 6),
 (1, 1),
 (7, 5),
 (7, 7),
 (6, 6),
 (7, 7),
 (5, 7),
 (5, 6),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (6, 1),
 (7, 7),
 (7, 7),
 (3, 3),
 (7, 7),
 (1, 1),
 (6, 6),
 (5, 4),
 (7, 1),
 (5, 5),
 (7, 5),
 (7, 1),
 (7, 5),
 (7, 7),
 (0, 5),
 (6, 6),
 (7, 6),
 (7, 7),
 (7, 7),
 (7, 7),
 (6, 6),
 (5, 5),
 (7, 1),
 (7, 7),
 (7, 3),
 (7, 7),
 (7, 6),
 (4, 4),
 (5, 5),
 (7, 7),
 (1, 4),
 (5, 5),
 (5, 5),
 (7, 5),
 (7, 7),
 (6, 5),
 (5, 5),
 (7, 7),
 (5, 6),
 (7, 7),
 (5, 7),
 (5, 4),
 (0, 5),
 (5, 4),
 (7, 7),
 (7, 0),
 (1, 0),
 (7, 5),
 (7, 7),
 (5, 7),
 (7, 5),
 (7, 7),
 (7, 7),
 (6, 4),
 (7, 5),
 (7, 7),
 (7, 7),
 (7, 5),
 (5, 4),
 (7, 6),
 (7, 7),
 (7, 7),
 (6, 5),
 (7, 6),
 (4, 5),
 (6, 0),
 (5, 2),
 (4, 6),
 (4, 1),
 (6, 7),
 (7, 7),
 

In [65]:
%%time
automl2 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60, resampling_strategy='holdout')
automl2.fit(X_test, y_test)

[INFO] [2016-02-04 12:39:25,998:AutoML(1):47420fc2015c48bbb12061d640fff6cf] Start calculating metafeatures for 47420fc2015c48bbb12061d640fff6cf
[INFO] [2016-02-04 12:39:26,006:AutoML(1):47420fc2015c48bbb12061d640fff6cf] Calculating Metafeatures (categorical attributes) took  0.01
[INFO] [2016-02-04 12:39:26,074:AutoML(1):47420fc2015c48bbb12061d640fff6cf] Calculating Metafeatures (encoded attributes) took  0.07sec
[INFO] [2016-02-04 12:39:26,551:AutoML(1):47420fc2015c48bbb12061d640fff6cf] Time left for 47420fc2015c48bbb12061d640fff6cf after finding initial configurations: 58.07sec
[INFO] [2016-02-04 12:39:26,556:autosklearn.util.smac] Start SMAC with 58.06sec time left
[INFO] [2016-02-04 12:39:26,557:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_19930_4015/47420fc2015c48bbb12061d640fff6cf.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0 --intensification-percentage 0.5 --validation false --

In [77]:
#y_full = train['Response'].values
#y_full = y_full-1

X_full = train[features].as_matrix()
final_preds = automl2.predict(X_full)

ValueError: operands could not be broadcast together with shape (8,) (126,)

In [23]:
#think about a few things here... first a large scale test, have it run for however long it wants to and 
#see what the results are
#second... using a bayesopt method to see best params for the autosklearn clf
#third... use several different models to blend together... xgboost + autolearn clf = 2 cols -> autolearn -> predict
#fourth ... continue using offsets?


pred_df = pd.DataFrame(preds, columns=['1','2','3','4','5','6','7','8'])

In [25]:
pred_df.to_csv('autosklearn_preds.csv')

In [36]:
# Hint:
weights = [0.05, 0.1, 0.85]
choices = ["five percent chance of me", "ten of me", "85 of me"]
print np.random.choice(choices, p=weights)
print [np.random.choice(choices, p=weights) for i in range(10)]

def softmax(q, a, tau):
    reward = q[a][0]
    trials = q[a][1]
    try:
        mean_reward = reward/trials
        sm = (np.exp(mean_reward/tau) / sum([np.exp((q[i][0]/q[i][1])/tau) for i in range(8)]))
    except ZeroDivisionError:
        sm = 0.
    return sm

85 of me
['85 of me', '85 of me', 'five percent chance of me', '85 of me', '85 of me', '85 of me', 'ten of me', '85 of me', '85 of me', '85 of me']


In [34]:
from collections import defaultdict
q = defaultdict(float)
type(q)
q = {action: [0.0, 0] for action in range(10)}

num_trials = 500
weights = [softmax(q, i, 1.5) for i in range(10)]
choices = range(10)
print weights, choices
print np.random.choice(choices, p=weights)
slot_list = [np.random.choice(choices, p=weights) for i in range(num_trials)]
plt.scatter(range(num_trials), slot_list)
plt.show()




[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


ValueError: probabilities do not sum to 1

collections.defaultdict

In [2]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]
X_train = X[:1000]
y_train = y[:1000]
X_test = X[1000:]
y_test = y[1000:]


In [6]:
autoclf = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=600)


In [7]:
%%time
autoclf.fit(X_train, y_train)
print(autoclf.score(X_test,y_test))

[INFO] [2016-02-03 14:52:04,756:AutoML(1):befcf1bed70ffd15ef397b08e29ee2c9] Start calculating metafeatures for befcf1bed70ffd15ef397b08e29ee2c9
[INFO] [2016-02-03 14:52:04,760:AutoML(1):befcf1bed70ffd15ef397b08e29ee2c9] Calculating Metafeatures (categorical attributes) took  0.00
[INFO] [2016-02-03 14:52:04,790:AutoML(1):befcf1bed70ffd15ef397b08e29ee2c9] Calculating Metafeatures (encoded attributes) took  0.03sec
[INFO] [2016-02-03 14:52:05,238:AutoML(1):befcf1bed70ffd15ef397b08e29ee2c9] Time left for befcf1bed70ffd15ef397b08e29ee2c9 after finding initial configurations: 598.25sec
[INFO] [2016-02-03 14:52:05,243:autosklearn.util.smac] Start SMAC with 598.24sec time left
[INFO] [2016-02-03 14:52:05,244:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_2918_7489/befcf1bed70ffd15ef397b08e29ee2c9.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0 --intensification-percentage 0.5 --validation false -

In [20]:
print(automl.score(X_test, y_test))

IndexError: index 8 is out of bounds for axis 1 with size 8

In [10]:
# -*- encoding: utf-8 -*-
from __future__ import print_function

import sklearn.datasets
import numpy as np

import autosklearn.classification


def main():
    digits = sklearn.datasets.load_digits()
    X = digits.data
    y = digits.target
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    X_train = X[:1000]
    y_train = y[:1000]
    X_test = X[1000:]
    y_test = y[1000:]
    automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=100,
                                               per_run_time_limit=30,
                                               tmp_folder='/tmp/autoslearn_example_tmp',
                                               output_folder='/tmp/autosklearn_example_out')
    automl.fit(X_train, y_train, dataset_name='digits')
    print(automl.score(X_test, y_test))


if __name__ == '__main__':
    main()

[INFO] [2016-02-02 21:07:21,347:AutoML(1):digits] Start calculating metafeatures for digits
[INFO] [2016-02-02 21:07:21,351:AutoML(1):digits] Calculating Metafeatures (categorical attributes) took  0.00
[INFO] [2016-02-02 21:07:21,378:AutoML(1):digits] Calculating Metafeatures (encoded attributes) took  0.03sec
[INFO] [2016-02-02 21:07:21,825:AutoML(1):digits] Time left for digits after finding initial configurations: 98.30sec
[INFO] [2016-02-02 21:07:21,830:autosklearn.util.smac] Start SMAC with 98.29sec time left
[INFO] [2016-02-02 21:07:21,831:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autoslearn_example_tmp/digits.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0 --intensification-percentage 0.5 --validation false --initial-challengers " -balancing:strategy 'weighting' -classifier:__choice__ 'sgd' -classifier:sgd:alpha '0.0001' -classifier:sgd:average 'True' -classifier:sgd:eta0 '0.01' -classifier:s

ValueError: No models fitted!

In [10]:
import pandas as pd
import numpy as np
import time
import random
from sklearn.cross_validation import StratifiedKFold
from scipy.optimize import fmin_powell
from ml_metrics import quadratic_weighted_kappa
import autosklearn.classification

clf = autosklearn.classification.AutoSklearnClassifier(resampling_strategy='holdout')
clfs = [[clf],[clf],[clf],[clf],[clf],[clf],[clf],[clf],[clf],[clf]]
columns = [['AutoML1'],['AutoML2'],['AutoML3'],['AutoML4'],['AutoML5'],['AutoML6'],['AutoML7'],['AutoML8'],['AutoML9'],['AutoML10']]



In [11]:
def train_and_cv(clfs, skf, X_dev, Y_dev, X_test, columns, n_folds):
    blend_train = np.zeros((X_dev.shape[0], 8)) #len(clfs))) # Number of training data x Number of classifiers
    blend_test = np.zeros((X_test.shape[0], 8)) #len(clfs))) # Number of testing data x Number of classifiers
    
    for j, clf in enumerate(clfs):
        print 'Training classifier [%s]' % (columns[j])
        blend_test_j = np.zeros((X_test.shape[0], 8*n_folds)) #len(skf))) # Number of testing data x Number of folds 
                                                             #, we will take the mean of the predictions later
        
        for i, (train_index, cv_index) in enumerate(skf):
            print 'Fold [%s]' % (i)
            # This is the training and validation set
            
            X_train = X_dev[train_index]
            X_cv = X_dev[cv_index]
                
            Y_train = Y_dev[train_index]
            Y_cv = Y_dev[cv_index]
            
            
            
            if "XGB" in columns[j]:
                dtrain=xgb.DMatrix(X_train,Y_train)
                dcv = xgb.DMatrix(X_cv, Y_cv)
                
                model = xgb.train(get_params(columns[j]), dtrain, 500)
                blend_train[cv_index, j] = model.predict(dcv, ntree_limit=model.best_iteration)
            
                dtest=xgb.DMatrix(X_test, label=Y_dev)
                blend_test_j[:,i] = model.predict(dtest, ntree_limit=model.best_iteration)
                
                

            elif "A" in columns[j]:
                clf.fit(X_train, Y_train)
                
                train_preds = clf.predict(X_cv)
                test_preds = clf.predict(X_test)
                
                train_preds_df = pd.DataFrame(train_preds, columns=['1','2','3','4','5','6','7','8'])
                test_preds_df = pd.DataFrame(test_preds, columns=['1','2','3','4','5','6','7','8'])
                
                
                for k in range(train_preds.shape[1]):
                    blend_train[cv_index, k] = train_preds_df[str(k+1)].values #insert appropriate model predictions here
                    
                    if i == 0:
                        blend_test_j[:, k] = test_preds_df[str(k+1)].values #insert appropriate model predictions here
                    elif i == 1:
                        blend_test_j[:, k+8] = test_preds_df[str(k+1)].values
                    elif i == 2:
                        blend_test_j[:, k+16] = test_preds_df[str(k+1)].values
                    elif i == 3:
                        blend_test_j[:, k+24] = test_preds_df[str(k+1)].values
                    elif i == 4:
                        blend_test_j[:, k+32] = test_preds_df[str(k+1)].values
                    #elif i == 5:
                    #    blend_test_j[:, k+40] = test_preds_df[str(k+1)].values
                    #elif i == 6:
                    #    blend_test_j[:, k+48] = test_preds_df[str(k+1)].values
                    #elif i == 7:
                    #    blend_test_j[:, k+56] = test_preds_df[str(k+1)].values
                    #elif i == 8:
                    #    blend_test_j[:, k+64] = test_preds_df[str(k+1)].values
                    #elif i == 9:
                    #    blend_test_j[:, k+72] = test_preds_df[str(k+1)].values
                        
                
            else:
                clf.fit(X_train, Y_train)
                # This output will be the basis for our blended classifier to train against,
                # which is also the output of our classifiers
                blend_train[cv_index, j] = clf.predict(X_cv)
                blend_test_j[:,i] = clf.predict(X_test)
                #print blend_test_j[:,i]
            
            
            
            
            


        # Take the mean of the predictions of the cross validation set... but how to blend w/ multiclass?
        
        #blend_test[:,j] = blend_test_j.mean(1)
        
    test_pred_avg = []
    test_pred_row_avg = []

    for row in blend_test_j:
        for i in range(8):
            avg = (row[i]+row[i+8]+row[i+16]+row[i+24]+row[i+32])/5
            test_pred_row_avg.append(avg)
        test_pred_avg.append(test_pred_row_avg)
        test_pred_row_avg = []
    test_pred_avg = np.array(test_pred_avg)
    
    return blend_train, test_pred_avg

In [12]:
%%time
#start_time = time.time()
    
#DATA_DIR = '/Users/patrickkennedy/Desktop'
train = pd.read_csv('train.csv')
train.sample(frac=1)

test = pd.read_csv('test.csv')
# combine train and test
all_data = train.append(test)

# Use -1 for any others
all_data.fillna(-1, inplace=True)
    
all_data['Product_Info_2'] = pd.factorize(all_data['Product_Info_2'])[0]

# fix the dtype on the label column
all_data['Response'] = all_data['Response'].astype(int)
print all_data.shape
#is this crossvalidated on train set or to build for submission?
for_reals = False
    

#add a real_deal boolean so that i don't have to recode when i use test/train vs kaggle submission
features = all_data.columns
features = features.drop("Id")
features = features.drop("Response")
    
if for_reals:

    train = all_data[all_data['Response']>0].copy()
    test = all_data[all_data['Response']<1].copy()
                
    X = train[features].as_matrix()
    init_y = train["Response"]-1
    Y = init_y.as_matrix()
        
    X_dev = X[:]
    Y_dev = Y[:]
    X_test = test[features]

else:
    all_data = all_data[all_data['Response']>0].copy()    
    X = all_data[features].as_matrix()
        
    init_y = all_data["Response"]-1
    Y = init_y.as_matrix()
        
    # The DEV SET will be used for all training and validation purposes
    # The TEST SET will never be used for training, it is the unseen set.
    dev_cutoff = len(Y) * 4/5
    X_dev = X[:dev_cutoff]
    Y_dev = Y[:dev_cutoff]
    X_test = X[dev_cutoff:]  #test[features]
    Y_test = Y[dev_cutoff:]  #test[dev_cutoff:]
        
n_folds = 5
# Ready for cross validation
skf = list(StratifiedKFold(Y_dev, n_folds))
# For each classifier, we train the number of fold times (=len(skf))
    
levels = 1
    
for i in range(levels):        
    blend_train, blend_test = train_and_cv(clfs[i], skf, X_dev, Y_dev, X_test, columns[i], n_folds)
    X_dev = blend_train
    X_test = blend_test
        
    
print blend_train.shape
print blend_train
print
print
print blend_test.shape
print blend_test
    
#return blend_train, blend_test
   
#blend_train_preds = blend_train.mean(1)
#blend_test_preds = blend_test.mean(1)
    
#blend_train_preds = blend_train_preds+1
#blend_test_preds = blend_test_preds+1
    
#int_train_preds = np.clip(blend_train_preds, -0.99, 8.99).astype(int)
#int_test_preds = np.clip(blend_test_preds, -0.99, 8.99).astype(int)
#splits = [1, 2.5, 3.5, 4, 5.2, 6.8, 7.5, 8]
#int_train_preds = np.digitize(blend_train_preds, splits).astype(int)
#int_test_preds = np.digitize(blend_test_preds, splits).astype(int)
    
#num_classes=8
#train offsets 
#offsets = np.ones(num_classes) * -0.5
#offset_train_preds = np.vstack((int_train_preds, int_train_preds, Y_dev))

#for j in range(num_classes):
#    train_offset = lambda x: -apply_offset(offset_train_preds, x, j)
#    offsets[j] = fmin_powell(train_offset, offsets[j])  

# apply offsets to test
#int_test_vals = (int_test_preds*0)-1
#data_off = np.vstack((int_test_preds, int_test_preds, int_test_vals))  #here we exchange test response with -1
#for j in range(num_classes):
#    data_off[1, data_off[0].astype(int)==j] = data_off[0, data_off[0].astype(int)==j] + offsets[j]    
#final_test_preds = np.round(np.clip(data_off[1], 1, 8)).astype(int)
#final_test_preds = np.digitize(data_off[1], splits).astype(int)
    
#if for_score:
#    preds_out = pd.DataFrame({"Id": test['Id'].values, "Response": final_test_preds})
#    preds_out = preds_out.set_index('Id')
#    preds_out.to_csv('ensemble.csv')
    
#else:
    # Predict now
#    score = eval_wrapper(final_test_preds, Y_test)
#    print 'Accuracy = %s' % (score)
    
#return final_test_preds, blend_train, blend_test #, score

(79146, 128)
Training classifier [AutoML1]
Fold [0]
[INFO] [2016-02-08 09:52:14,054:AutoML(1):ad5d6631570445882e7b2fe7dd101c09] Start calculating metafeatures for ad5d6631570445882e7b2fe7dd101c09
[INFO] [2016-02-08 09:52:14,107:AutoML(1):ad5d6631570445882e7b2fe7dd101c09] Calculating Metafeatures (categorical attributes) took  0.05
[INFO] [2016-02-08 09:52:14,659:AutoML(1):ad5d6631570445882e7b2fe7dd101c09] Calculating Metafeatures (encoded attributes) took  0.55sec
[INFO] [2016-02-08 09:52:15,132:AutoML(1):ad5d6631570445882e7b2fe7dd101c09] Time left for ad5d6631570445882e7b2fe7dd101c09 after finding initial configurations: 3596.38sec
[INFO] [2016-02-08 09:52:15,139:autosklearn.util.smac] Start SMAC with 3596.37sec time left
[INFO] [2016-02-08 09:52:15,139:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_3655_9672/ad5d6631570445882e7b2fe7dd101c09.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0

KeyboardInterrupt: 

In [4]:
index_max=[]
for i in range(blend_test.shape[0]):
    index_max.append(blend_test[i].argmax())

In [5]:
index_max = np.array(index_max)

In [6]:
X.shape, X_dev.shape, X_test.shape, X.shape[0]==X_dev.shape[0]+X_test.shape[0]

((59381, 126), (47504, 8), (11877, 8), True)

In [7]:
Y.shape, Y_dev.shape, Y_test.shape


((59381,), (47504,), (11877,))

In [8]:
quadratic_weighted_kappa(index_max, Y_test)

0.39301212045583533

In [9]:
blend_test[2]

array([ 0.12268978,  0.12513861,  0.11767692,  0.11907522,  0.11990633,
        0.12450605,  0.12457541,  0.14643168])

In [10]:
blend_train[0]

array([ 0.12439183,  0.12554414,  0.12020298,  0.1206193 ,  0.12274631,
        0.1292939 ,  0.12559838,  0.13160318])

In [11]:
#turn these into DFs
blend_train_df = pd.DataFrame(blend_train, columns=['1','2','3','4','5','6','7','8'])
blend_test_df = pd.DataFrame(blend_test, columns=['1','2','3','4','5','6','7','8'])

blend_train_df['Response'] = Y_dev
blend_test_df['Response'] = Y_test

blend_train_df.to_csv('blend_train.csv')
blend_test_df.to_csv('blend_test.csv')


In [1]:
import pandas as pd
import numpy as np
import autosklearn.classification
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from ml_metrics import quadratic_weighted_kappa

In [2]:
offset = pd.read_csv("offset.csv")

In [3]:
offset_df = pd.DataFrame(offset)

In [4]:
features = ["First_Off", "Second_Off", "Third_Off", "Fourth_Off", "Fifth_Off", "Sixth_Off", "Seventh_Off", "Eighth_Off"]

In [5]:
train = offset_df[features]
labels = offset_df['Outcome']
full_data = pd.concat([train, labels],axis=1)


from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.33, random_state=42)

In [6]:
model = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60)

In [8]:

model.fit(X_train.as_matrix(), y_train.values)

[INFO] [2016-02-08 07:33:21,127:autosklearn.evaluation.resampling] Too few samples of one class or maybe a regression dataset, use shuffle split.

[INFO] [2016-02-08 07:33:22,542:AutoML(1):07fb5a1e97aaf2c109f27b5a9ac41210] Start calculating metafeatures for 07fb5a1e97aaf2c109f27b5a9ac41210
[INFO] [2016-02-08 07:33:22,550:AutoML(1):07fb5a1e97aaf2c109f27b5a9ac41210] Calculating Metafeatures (categorical attributes) took  0.01
[INFO] [2016-02-08 07:33:22,658:AutoML(1):07fb5a1e97aaf2c109f27b5a9ac41210] Calculating Metafeatures (encoded attributes) took  0.11sec
[INFO] [2016-02-08 07:33:23,113:AutoML(1):07fb5a1e97aaf2c109f27b5a9ac41210] Time left for 07fb5a1e97aaf2c109f27b5a9ac41210 after finding initial configurations: 3598.00sec
[INFO] [2016-02-08 07:33:23,118:autosklearn.util.smac] Start SMAC with 3598.00sec time left
[INFO] [2016-02-08 07:33:23,119:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_3655_577/07fb5a1e97aaf2c109f27b5a9ac41210.scenario --num-ei-

/home/patrick/anaconda2/envs/autolearn/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1136: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)
/home/patrick/anaconda2/envs/autolearn/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/patrick/anaconda2/envs/autolearn/lib/python2.7/site-packages/sklearn/cross_validation.py:417: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
